In [ ]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import logging
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from google.colab import userdata

# Alpaca API credentials
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"  # For paper trading
APCA_API_KEY_ID = userdata.get('APCA_API_KEY_ID')
APCA_API_SECRET_KEY = userdata.get('APCA_API_SECRET_KEY')

# Initialize Alpaca API
api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, base_url=APCA_API_BASE_URL, api_version='v2')

# Trading parameters
symbols = ["AAPL"]  # Replace with the desired stock symbols
timeframe = "1Min"  # Minute timeframe for high-frequency trading
short_window = 50  # Short moving average window
long_window = 200  # Long moving average window
trade_frequency = 1000  # Number of trades per day

# Position sizing and risk management
risk_per_trade = 0.02  # Risk 2% of the account per trade
portfolio_value = api.get_account().cash
risk_amount = portfolio_value * risk_per_trade

# Machine learning parameters
training_window = 20  # Window size for feature calculation
prediction_window = 5  # Number of future data points to predict

# Different machine learning models
model_for_strategy = RandomForestClassifier(n_estimators=100, random_state=42)
model_for_condor_vs_butterfly = SVC(kernel='linear', random_state=42)

# Setup logging
logging.basicConfig(filename='trading_log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to check if the short moving average crosses above the long moving average
def is_crossover(data):
    short_ma = data['close'].rolling(window=short_window).mean()
    long_ma = data['close'].rolling(window=long_window).mean()

    return short_ma.iloc[-1] > long_ma.iloc[-1] and short_ma.iloc[-2] <= long_ma.iloc[-2]

# Function to check if the short moving average crosses below the long moving average
def is_crossunder(data):
    short_ma = data['close'].rolling(window=short_window).mean()
    long_ma = data['close'].rolling(window=long_window).mean()

    return short_ma.iloc[-1] < long_ma.iloc[-1] and short_ma.iloc[-2] >= long_ma.iloc[-2]

# Function to extract features for machine learning
def extract_features(data):
    data['short_ma'] = data['close'].rolling(window=short_window).mean()
    data['long_ma'] = data['close'].rolling(window=long_window).mean()
    data['return'] = data['close'].pct_change()
    data['volatility'] = data['return'].rolling(window=training_window).std()

    # Shift the data to predict the future
    data['target'] = np.where(data['close'].shift(-prediction_window) > data['close'], 1, 0)

    return data.dropna()

# Train the machine learning model for strategy prediction
def train_model_for_strategy(data):
    X = data[['short_ma', 'long_ma', 'return', 'volatility']]
    y = data['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_for_strategy.fit(X_train, y_train)

    predictions = model_for_strategy.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    logging.info(f"Strategy Prediction Model Accuracy: {accuracy}")

# Train the machine learning model for Condor vs. Butterfly prediction
def train_model_for_condor_vs_butterfly(data):
    # Replace 'some_option_related_features' with actual option-related features
    X = data[['some_option_related_features']]
    y = data['target']  # Assuming you have a target variable (1 for Condor, 0 for Butterfly)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_for_condor_vs_butterfly.fit(X_train, y_train)

    predictions = model_for_condor_vs_butterfly.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    logging.info(f"Condor vs. Butterfly Model Accuracy: {accuracy}")

# Function to predict the best strategy
def predict_best_strategy(data):
    # Replace 'some_option_related_features' with actual option-related features
    features_for_condor_vs_butterfly = data[['some_option_related_features']]

    # Predict using the SVM model
    prediction = model_for_condor_vs_butterfly.predict(features_for_condor_vs_butterfly)[0]

    if prediction == 1:
        return 'iron_condor'
    else:
        return 'butterfly_spread'

# Function to check if Iron Condor strategy conditions are met
def is_iron_condor(data):
    # Implement Iron Condor conditions (example: out-of-the-money call and put options)
    return True  # Replace with actual conditions

# Function to check if Butterfly Spread strategy conditions are met
def is_butterfly_spread(data):
    # Implement Butterfly Spread conditions (example: combination of call and put options)
    return True  # Replace with actual conditions

# Function to execute Iron Condor strategy
def execute_iron_condor(symbol):
    # Implement Iron Condor strategy actions
    logging.info(f"Iron Condor strategy executed for {symbol}")

# Function to execute Butterfly Spread strategy
def execute_butterfly_spread(symbol):
    # Implement Butterfly Spread strategy actions
    logging.info(f"Butterfly Spread strategy executed for {symbol}")

# Main trading function
def trade():
    for symbol in symbols:
        # Get historical data
        historical_data = api.get_barset(symbol, timeframe, limit=long_window).df[symbol]

        # Extract features for machine learning
        features_data = extract_features(historical_data.copy())

        # Train the machine learning models
        train_model_for_strategy(features_data)
        train_model_for_condor_vs_butterfly(features_data)

        # Simulate high-frequency trading
        for _ in range(trade_frequency):
            # Make predictions using the machine learning model for strategy
            current_data = extract_features(historical_data.copy().tail(training_window))
            strategy_prediction = model_for_strategy.predict(current_data[['short_ma', 'long_ma', 'return', 'volatility']].iloc[-1].values.reshape(1, -1))[0]

            # Fine-tune strategies based on machine learning predictions
            strategies = []

            # Example strategies
            strategies.append({
                'name': 'Moving Average Crossover',
                'condition': is_crossover(historical_data),
                'action': 'buy',
            })

            strategies.append({
                'name': 'Moving Average Crossunder',
                'condition': is_crossunder(historical_data),
                'action': 'sell',
            })

            # Determine the best strategy using SVM prediction
            best_strategy = predict_best_strategy(current_data)

            # Execute trading actions based on strategies
            for strategy in strategies:
                if strategy['condition']:
                    if strategy['action'] == 'buy':
                        # Calculate position size based on risk
                        stop_loss_price = historical_data['low'].min()
                        position_size = int(risk_amount / (historical_data['close'].iloc[-1] - stop_loss_price))

                        # Execute buy order
                        api.submit_order(
                            symbol=symbol,
                            qty=position_size,
                            side='buy',
                            type='market',
                            time_in_force='gtc'
                        )
                        logging.info(f"Buy order executed for {symbol} - Quantity: {position_size}, Entry Price: {historical_data['close'].iloc[-1]}")

                    elif strategy['action'] == 'sell':
                        # Get current position and sell all shares
                        position = api.get_position(symbol)
                        if position:
                            # Execute sell order
                            api.submit_order(
                                symbol=symbol,
                                qty=position.qty,
                                side='sell',
                                type='market',
                                time_in_force='gtc'
                            )
                            logging.info(f"Sell order executed for {symbol} - Quantity: {position.qty}, Exit Price: {historical_data['close'].iloc[-1]}")

                    # Implement options trading strategies based on machine learning predictions
                    if best_strategy == 'iron_condor' and is_iron_condor(current_data):
                        execute_iron_condor(symbol)

                    elif best_strategy == 'butterfly_spread' and is_butterfly_spread(current_data):
                        execute_butterfly_spread(symbol)

if __name__ == "__main__":
    while True:
        trade()
        time.sleep(60)  # Sleep for 1 minute before checking again for high-frequency trading
